In [ ]:
import speech_recognition as sr
import pandas as pd
from textblob import TextBlob
from googletrans import Translator
import sys
from PyQt5 import uic
from PyQt5.QtWidgets import QMainWindow, QApplication
import mysql.connector as mariadb
mariadb_connection = mariadb.connect(user='root', password='Garibaldi', database='nlp')
cur = mariadb_connection.cursor()

class clasificador(QMainWindow):
    
    def __init__(self):
        super().__init__()
        uic.loadUi("inicio.ui", self)
        self.BtnCalificar.clicked.connect(self.grabar)
        self.BtnEstadistica.clicked.connect(self.estadistica)
        self.BtnSalir.clicked.connect(self.salir)

    def grabar(self):
        r=sr.Recognizer()
        with sr.Microphone() as recurso:
            print('Grabe su opinión sobre el producto')
            audio=r.listen(recurso)
    
            try:
                texto=r.recognize_google(audio, language='es-ES')
                txt=r.recognize_google(audio, language='es-ES')
                print("Muchas gracias por sus comentarios :{}".format(texto))
            except Exception as excepcion:
                print("Lo siento no entendí, por favor intente nuevamente")
                print(str(excepcion))
        
        translation=Translator(service_urls=["translate.google.com"])
        textEnglish=translation.translate(texto,dest="en")
        print(textEnglish,texto)
        texto=textEnglish.text
                
        edu=TextBlob(texto)
        valor=edu.sentiment.polarity
        
        if valor<0:
            print('Calificación Negativa ' + str(valor))
            self.Lbl_mensaje.setText('CALIFICACION NEGATIVA.\n\n Valor: ==>   ' + str(round(valor,4)) + '\n\n' + txt)
        elif valor==0:
            print('Calificación Neutra ' + str(valor))
            self.Lbl_mensaje.setText('CLASIFICACION NEUTRAL.\n\n Valor: ==>   ' + str(round(valor,4)) + '\n\n' + txt)
        elif valor>0 and valor<=1:
            print('Calificación Positiva ' + str(valor))
            self.Lbl_mensaje.setText('CLASIFICACION POSITIVA.\n\n Valor: ==>   ' + str(round(valor,4)) + '\n\n' + txt)

        if(self.RB_Tv.isChecked()):
            with open("audio.wave","wb") as archivo:
                cod=100
                print(str(cod) + '-----------TELEVISOR')
        if self.RB_Nevecon.isChecked():
            with open("audio.wave","wb") as archivo:
                cod=200
                print(str(cod) + '-----------NEVECON')
        if self.RB_Lavadora.isChecked():
            with open("audio.wave","wb") as archivo:
                cod=300
                print(str(cod) + '-----------LAVADORA')
        
        print(valor)
        print(txt)
        print(cod)
       
        cur.execute("INSERT INTO calificacion (valor, textValor, FK_idProducto) VALUES (%s,%s,%s)", (valor, txt, cod))
        mariadb_connection.commit()
        #cur.close()
        #mariadb_connection.close()

    def estadistica(self):
        if(self.RB_Tv.isChecked()):
            cur.execute("SELECT COUNT(*) FROM calificacion WHERE (FK_idProducto=%s AND valor>%s)", ('100', '0'))
            posit=cur.fetchall()
            cur.execute("SELECT COUNT(*) FROM calificacion WHERE (FK_idProducto=%s AND valor=%s)", ('100', '0'))
            neut=cur.fetchall()
            cur.execute("SELECT COUNT(*) FROM calificacion WHERE (FK_idProducto=%s AND valor<%s)", ('100', '0'))
            negativ=cur.fetchall()
            self.Lbl_mensaje.setText('CALIFICACIONES REGISTRADAS AL TELEVISOR \n\n Positivas: ==>   ' + str(posit) + '\n\n Neutras:   ==>   ' + str(neut) + '\n\n Negativas: ==>  ' + str(negativ) )
        if self.RB_Nevecon.isChecked():
            cur.execute("SELECT COUNT(*) FROM calificacion WHERE (FK_idProducto=%s AND valor>%s)", ('200', '0'))
            posit=cur.fetchall()
            cur.execute("SELECT COUNT(*) FROM calificacion WHERE (FK_idProducto=%s AND valor=%s)", ('200', '0'))
            neut=cur.fetchall()
            cur.execute("SELECT COUNT(*) FROM calificacion WHERE (FK_idProducto=%s AND valor<%s)", ('200', '0'))
            negativ=cur.fetchall()
            self.Lbl_mensaje.setText('CALIFICACIONES REGISTRADAS AL NEVECON  \n\n Positivas: ==>   ' + str(posit) + '\n\n Neutras:   ==>   ' + str(neut) + '\n\n Negativas: ==>  ' + str(negativ) )        
        if self.RB_Lavadora.isChecked():
            cur.execute("SELECT COUNT(*) FROM calificacion WHERE (FK_idProducto=%s AND valor>%s)", ('300', '0'))
            posit=cur.fetchall()
            cur.execute("SELECT COUNT(*) FROM calificacion WHERE (FK_idProducto=%s AND valor=%s)", ('300', '0'))
            neut=cur.fetchall()
            cur.execute("SELECT COUNT(*) FROM calificacion WHERE (FK_idProducto=%s AND valor<%s)", ('300', '0'))
            negativ=cur.fetchall()
            self.Lbl_mensaje.setText('CALIFICACIONES REGISTRADAS A LA LAVADORA \n\n Positivas: ==>   ' + str(posit) + '\n\n Neutras:   ==>   ' + str(neut) + '\n\n Negativas: ==>  ' + str(negativ) )
        
        #cur.close()
        #mariadb_connection.close()

    def salir(self):
        sys.exit()
    
if __name__ == '__main__':
    app = QApplication(sys.argv)
    GUI = clasificador()
    GUI.show()
    sys.exit(app.exec_())
